Insert Packages 

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install lxml

In [3]:
!pip install html5lib

In [4]:
!pip install requests

# Part 1

Import pandas, numpy, and BeautifulSoup

In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [6]:
#get wiki link
wiki_link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
#use BeautifulSoup to scrape the wiki page
soup = BeautifulSoup(wiki_link, 'html5lib')

In [8]:
#create a table
table1 = soup.find('table',{'class':'wikitable sortable'})
data1 = table1.find_all('td')

In [9]:
#append the wiki page data to the table
counter = 0
table = []
row = []
for details in data1:
    if counter < 3:
        row.append(details.get_text().strip())
        counter+=1
    else:
        table.append(row)
        counter = 0
        row = []
        row.append(details.get_text().strip())
        counter+=1
        
#define headers 
headers=['PostalCode','Borough','Neighbourhood']
df = pd.DataFrame(table, columns=headers)

#clean the data 
def cleaning (row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    return row
df1 = df.apply(cleaning, axis=1)
df1 = df1[df1['Borough'] != 'Not assigned']

#combine neighbourhoods with same postal code in one row 
df1 = df1.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [10]:
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
df1.shape

(103, 3)

# Part 2

In [12]:
!pip install geocoder

     |████████████████████████████████| 102kB 16.7MB/s ta 0:00:01


In [13]:
import geocoder # import geocoder

def get_coordinates1 (row):
    # initialize your variable to None
    lat_lng_coords = None
    print(row['Postcode'])
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
        lat_lng_coords = g.latlng
        print('1')

    row['Latitude'] = lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]
    print(row['Latitude'], row['Longitude'])
    return row


In [14]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f389ca35935b4f8e982b864138e3fe53 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9qd8J6nujvR2eD3BYsZsF0MVx8A3hChJWtqNAPuKw0mv',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_f389ca35935b4f8e982b864138e3fe53.get_object(Bucket='capstoneweek1-donotdelete-pr-q6fsoogtqc1ljz',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

coordinates_df = pd.read_csv(body)
coordinates_df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
def get_coordinates2 (row):
    temperature_df = coordinates_df[coordinates_df['Postal Code'] == row['PostalCode']]
    row['Latitude'] = float(temperature_df['Latitude'])
    row['Longitude'] = float(temperature_df['Longitude'])
    return row
    
df_with_coordinates = df1
df_with_coordinates['Latitude'] = np.nan
df_with_coordinates['Longitude'] = np.nan
df_with_coordinates = df_with_coordinates.apply(get_coordinates2, axis=1)
df_with_coordinates.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Part 3

In [16]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

Create a folium map using the Toronto dataframe 

In [17]:
import folium
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_with_coordinates['Latitude'], df_with_coordinates['Longitude'], df_with_coordinates['Borough'], df_with_coordinates['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough.upper())
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Foursquare Client ID/Secret

In [43]:
CLIENT_ID = '' #'client-ID' Foursquare ID
CLIENT_SECRET = '' #  Foursquare Secret
VERSION = '20180605'
print('My Credentails:')
print('CLIENT_ID: ' )
print('CLIENT_SECRET:')

My Credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [44]:
df_toronto_neighbourhoods = df_with_coordinates 
df_toronto_neighbourhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Get Nearby Venues 

In [45]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
df_toronto_venues = getNearbyVenues(names=df_toronto_neighbourhoods['Neighbourhood'],
                                   latitudes=df_toronto_neighbourhoods['Latitude'],
                                   longitudes=df_toronto_neighbourhoods['Longitude'])

Rouge, Malvern


KeyError: 'groups'

check the size of df

In [42]:
print(df_toronto_venues.shape)
df_toronto_venues.head()

NameError: name 'df_toronto_venues' is not defined

How many venues returned for each neighbourhood